<a href="https://colab.research.google.com/github/preetamjumech/DL_DNN_from_Scratch/blob/main/Preetam_Saha_28_09_2022_Regression__DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import os
os.environ['KAGGLE_CONFIG_DIR']="/content/drive/MyDrive/kaggle"

In [23]:
!kaggle datasets list -s "boston"

ref                                         title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
AnalyzeBoston/crimes-in-boston              Crimes in Boston                                     10MB  2018-09-04 17:56:03          38973        666  0.8235294        
schirmerchad/bostonhoustingmlnd             Boston Housing                                        4KB  2017-06-11 15:07:11          16831        182  0.8235294        
vikrishnan/boston-house-prices              Boston House Prices                                  13KB  2017-08-03 17:06:12          69839        450  0.8235294        
ankkur13/boston-crime-data                  Crimes in Boston                                     11MB  2018-10-04 13:52:08           9493        191  0.8235294 

In [24]:
!kaggle datasets download -d "fedesoriano/the-boston-houseprice-data"

the-boston-houseprice-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [25]:
!unzip "/content/the-boston-houseprice-data.zip"

Archive:  /content/the-boston-houseprice-data.zip
replace boston.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams["figure.figsize"]=(20,10)

In [27]:
data = pd.read_csv("/content/boston.csv")
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [28]:
data.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


In [30]:
x = data.drop("MEDV",axis=1)

y = data.iloc[:,-1]

In [31]:
x.shape, y.shape

((506, 13), (506,))

In [32]:
import torch

In [33]:
x = torch.tensor(x.values)
y = torch.tensor(y.values)

In [34]:
x.shape, y.shape

(torch.Size([506, 13]), torch.Size([506]))

In [82]:
y = y.reshape(-1,1)
y.shape

torch.Size([506, 1])

In [35]:
import torch,torchsummary
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import DataLoader,TensorDataset 
from torch.optim import Adam,RMSprop

In [83]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.3)

In [84]:
x_train.shape,  x_test.shape, y_train.shape, y_test.shape

(torch.Size([354, 13]),
 torch.Size([152, 13]),
 torch.Size([354, 1]),
 torch.Size([152, 1]))

In [85]:
train_loader = DataLoader(TensorDataset(x_train.float(),y_train),
                                       batch_size=64,shuffle = True)

test_loader = DataLoader(TensorDataset(x_test.float(),y_test),
                                      batch_size = 64,shuffle = False)

In [86]:
class DNN_MNIST(nn.Module):
  def __init__(self):
    super(DNN_MNIST,self).__init__()

    self.h1 = nn.Linear(in_features=13, out_features = 512)
    self.h2 = nn.Linear(in_features=512, out_features = 256)
    self.h3 = nn.Linear(in_features=256, out_features = 128)
    self.h4 = nn.Linear(in_features=128, out_features = 64)
    self.out = nn.Linear(in_features=64, out_features = 1)
    self.bn1 = nn.BatchNorm1d(num_features=512)
    self.bn2 = nn.BatchNorm1d(num_features=256)
    self.bn3 = nn.BatchNorm1d(num_features=128)
    self.dropout = nn.Dropout(p=0.4)

  def forward(self,x):
    x = F.relu(self.h1(x))
    x = self.bn1(x)
    x = self.dropout(x)

    x = F.relu(self.h2(x))
    x = self.bn2(x)
    x = self.dropout(x)

    x = F.relu(self.h3(x))
    x = self.bn3(x)
    x = self.dropout(x)

    x = F.relu(self.h4(x))
    x = self.dropout(x)

    x = self.out(x)
    return x

In [87]:
dnn_model = DNN_MNIST()

In [ ]:
list(dnn_model.parameters())

In [89]:
opt = Adam(dnn_model.parameters(),lr=0.001)

In [90]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [91]:
dnn_model = dnn_model.to(device)

In [92]:
from tqdm import tqdm

In [104]:
def training(model, train_loader, opt):
    train_loss = 0.0
    model.train()
    for batch, target in tqdm(train_loader):
        batch = batch.float().to(device)
        target = target.float().to(device)
        #print(target.dtype)

        opt.zero_grad()
        output = model(batch.float())
        #print(output.dtype)

        loss = F.mse_loss(output, target)
        loss.backward()
        opt.step()
        train_loss += loss.item()

    return train_loss

In [105]:
def validate(model, test_loader, opt):
    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for batch, target in tqdm(test_loader):
            batch = batch.float().to(device)
            target = target.float().to(device)

            output = model(batch.float())
            loss = F.mse_loss(output, target)
            val_loss += loss.item()
    return val_loss

In [122]:
model = dnn_model.to(device)
hist = {'train_loss':[], 'val_loss':[]}
for epoch in range(25):
    hist['train_loss'].append(training(model, train_loader,opt))
    hist['val_loss'].append(validate(model, test_loader,opt))

100%|██████████| 3/3 [00:00<00:00, 317.46it/s]


In [123]:
hist["train_loss"][-1] , hist["val_loss"][-1]

(223.65508651733398, 12969.545949935913)

In [124]:
final_pred = dnn_model(x_test.float())

In [125]:
from sklearn.metrics import r2_score

In [126]:
print(r2_score(y_test,final_pred.detach().numpy()))

-70.7634725730419
